# 1º - Importando bibliotecas:

In [117]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# 2º - Carregando datasets: 

In [118]:
cabecalho = ['Id','Texto','Sentimento']
dadosPositivos = pd.read_csv('https://raw.githubusercontent.com/matheusf06/repo_test/master/dataset-master/positivos.csv', sep=';', index_col = 0, names=cabecalho, encoding='utf-8')

In [119]:
cabecalho = ['Id','Texto','Sentimento']
dadosNegativos = pd.read_csv('https://raw.githubusercontent.com/matheusf06/repo_test/master/dataset-master/negativos2.csv', sep=',', names=cabecalho, index_col = 0)

# 3º - Apresentando DadosNegativos

In [120]:
dadosNegativos

,Texto,Sentimento
Id,,
id,text,sentiment
236,Morreu o moço latino americano :(,0
237,"Tt de cait,não confio :(",0
238,Acabei de arrumar a casa agora :(,0
239,"triste demais pela lindsay, mulher nenhuma mer...",0
...,...,...
17439,@espinosagozado Então :(,0
17440,@271CN Sério?! Mas por que? Tem vergonha? :(,0
17441,Quero meu pirscing nos peitos :(,0


# 4 - Apresentando DadosPositivos

In [121]:
dadosPositivos

,Texto,Sentimento
Id,,
id,text,sentiment
1,@caprichOreality Fica assim não miga &lt;3 Tud...,1
2,Parti me todo a descer a avenida de Gaia com o...,1
3,Amanhã é dia de dar um trato na palestra para ...,1
4,@thankovsky @patorebaichado eu também tenho :)...,1
...,...,...
8702,@Dg_Oliveer13 Segue de volta ou Unfollow :),1
8703,Criar pavão deve ser legal :),1
8704,@nataliakreuser Mas acho legal que os fãs de q...,1


# 5 - Cocatenando Dados

In [122]:
dados = pd.concat([dadosPositivos, dadosNegativos])
dados

,Texto,Sentimento
Id,,
id,text,sentiment
1,@caprichOreality Fica assim não miga &lt;3 Tud...,1
2,Parti me todo a descer a avenida de Gaia com o...,1
3,Amanhã é dia de dar um trato na palestra para ...,1
4,@thankovsky @patorebaichado eu também tenho :)...,1
...,...,...
17439,@espinosagozado Então :(,0
17440,@271CN Sério?! Mas por que? Tem vergonha? :(,0
17441,Quero meu pirscing nos peitos :(,0


# 6º - Funções para tratar os dados 

In [123]:
def remove_stopwords(text):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    words = [i for i in text.split() if not i in stopwords]
    return (" ".join(words))

def remove_links(text):
    return re.sub(r"http\S+", "", text)

def remove_mentions(text):
    return re.sub(r"@\w+", "", text)

def remove_retweets(text):
    return re.sub(r"rt\s", "", text)

def remove_special_chars(text):
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text_with_no_special_chars = re.sub("\s+", " ", text)
    return text_with_no_special_chars

def stemming(text):
    stemmer = nltk.stem.RSLPStemmer()
    words = []
    for word in text.split():
        words.append(stemmer.stem(word))
    return (" ".join(words))

def standardize_text(text):
    text = text.lower()
    text = remove_links(text)
    text = remove_mentions(text)
    text = remove_retweets(text)
    text = remove_stopwords(text)
    text = remove_special_chars(text)
    #text = stemming(text)
    return text
def sentiment_to_name(n):
    if n == 0:
        n = 'Negativo'
    elif n == 1:
        n = 'Positivo'
    return n

def name_to_sentiment(n):
    if n == 'Negativo':
        n = "0"
    elif n == 'Positivo':
        n = "1"
    return n

In [80]:
dados.Texto = dados.Texto.apply(standardize_text)
dados.Sentimento = dados.Sentimento.apply(sentiment_to_name)
dados.head(20)

,Sentimento,Texto
Id,,
id,sentiment,text
1,1,fica assim miga lt tudo arranja deus quiser
2,1,parti todo descer avenida gaia skate
3,1,amanhã dia dar trato palestra thedevconf ajust...
4,1,posso sentar vocês
5,1,ok sim aham tá boa vai lá
6,1,bila brao go firsetek çêbú serçava
7,1,prefiro amar clube vitórias ainda nao ganhamos sl
8,1,recomendamos vá loja claro acesse site confira...


# 7º - Verificando e excluindo dados nulos

In [124]:
dados.isnull().sum()

Texto         0
Sentimento    6
dtype: int64

In [125]:
dados = dados.dropna(how='any', axis=0)

In [126]:
dados.isnull().sum()

Texto         0
Sentimento    0
dtype: int64

# 8º - Classificando e testando a base de dados

In [127]:
dados_text = dados["Texto"].values
classes = dados['Sentimento'].values

In [128]:
vectorizer = CountVectorizer(ngram_range = (1, 2))
freq_tweets = vectorizer.fit_transform(dados_text)

modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [129]:
testes = ["bom",
          "ruim",
          "Odeio a politica brasileira",
          "final de período tá complicado",
          "Amizade",
          "link ruim"]

freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array(['1', '0', '0', '0', '0', '0'], dtype='<U9')